# Gradient Descent/Optimization Techniques

## Jacobian
The Jacobian matrix stores all of the first order partial derivatives of a vector valued function.
(Reminder: vector valued function = a function with multiple output values [into a vector]).

E.g.,

$f(x,y) = \begin{equation}
\left[
  \begin{array}{cccc}
  x^2y\\
  5x + sin y
  \end{array}
\right]
\end{equation}$

Then we can break it down into:

$f_1(x,y) = x^2y$

and

$f_2(x,y) = 5x + sin y$

Then the Jacobian matrix becomes:

$J_f(x,y) = \begin{equation}
\left[
  \begin{array}{cccc}
  \frac{\delta{f_1}}{\delta{x}} \frac{\delta{f_1}}{\delta{y}}\\
  \frac{\delta{f_2}}{\delta{x}} \frac{\delta{f_2}}{\delta{y}}
  \end{array}
\right]
\end{equation}$

$J_f(x,y) = \begin{equation}
\left[
  \begin{array}{cccc}
  2xy  &x^2\\
  5 &cos y
  \end{array}
\right]
\end{equation}$

The __Hessian__ matrix is similar, but it stores the second order partial derivatives (the derivative of the derivative).

## Gradient Descent

The gradient descent training process is as follows:

1. Randomly choose some weights for your network
2. Perform a forward pass on your data to get predictions for your data
3. Use a loss function to calculate how well your network is performing. The loss function will calculate some measure of difference between your predictions and the correct values. The lower the loss function, the better.
4. Calculate the gradient: this is the derivative of the cost function. Recall that a gradient will give you the slope of steepest ascent.
5. Update your weights by doing `weights - (gradient*learning rate)`. This will minimize the loss function, since we're moving "downhill", and so improve our weights. (Make the weights closer to values that will give more accurate predictions.)
6. Repeat steps 2-5
7. Once we have processed all of our data, repeat 2-6 for another epoch, and so on until we have good weights

#### "Stochastic"

What makes gradient descent __stochastic__ is that we evaluate our loss (aka error or cost) using just a subset of a data -- a mini-batch. This is because each update is a _stochastic approximation_ of the "true" cost gradient, and as as result the path down the function may "zig zag" a bit, but it should still converge at a minima as long as the function is convex.

#### Backpropagation

Since our cost function involves calculating a prediction by using the various activation functions in our network (weighted sums, ReLU, Sigmoid, etc.), taking the derivative of this function will necessarily involve taking the derivative of our entire network. In practical terms, this is implemented by treating of each activation as a "gate" in a "circuit". When we're doing our forward pass, we also calculate the derivative of each "gate" and then use this derivative during the backward pass. During the backward pass, we just keep applying the chain rule to work out what effect this gate (mathematical operation) has on the final output.

This lecture has a good walkthrough of the process: https://www.youtube.com/watch?v=i94OvYb6noo.

### Gradient Descent in Excel
`(See graddesx.xlsm)`

Imagine we have the simple function $y = ax + b$ ($a = slope, b = constant$).
We initialize them so $a=2$ and $b=30$ and generate some data. 

In "real" machine learning, we can think of $x$ and $y$ as our training data. For example, $x$ might be a picture of a cat or dog, and $y$ could be a label saying cat or dog. We want to calculate the correct value for $a$ and $b$ such that it predicts the label correctly.

Now, to do our learning, let's pretend we don't know the values for $a$ and $b$. Just initialize them to 1.

1. Get a prediction for y using our formula $y = ax + b$ (in a neural network, this formula could be the throughput of all the layers -- the weighted sum of a linear linear > ReLU > Softmax, etc.)
2. Calculate the error for our function by comparing the actual value and predicted value. For example, $(actual_y_value - predicted_y_value) ^2$

<img src="https://raw.githubusercontent.com/pekoto/fast.ai/master/images/gd1.jpg" width=650 height=500>

Now, how do we improve our weights? Notice that the cost function (step 2 above) will be closer to 0 when the prediction is closer to the actual value. So we want to minimize this cost function. To minimize a function, we can:

1. Calculate the gradient of the function (the slope of steepest ascent)
2. Step "down" the gradient by some learning rate

(I think there might be a slightly mistake in the Excel file here, as it assumes the cost function is predicted - actual, which flips a sign on the derivative, but it still works)

So, we can write our cost (error) function as:

$((ax+b)-y)^2$

($ax+b$ is our prediction, $y$ is our actual value).

If we calculate the partial derivatives for this...

The change in our function with respect to $b$ gives us:

$\frac{\delta{e}}{\delta{b}} = 2(ax+b-y)$

And with $a$:

$\frac{\delta{e}}{\delta{a}} = 2x(ax+b-y)$

So, to update our weights, we calulate the gradients using this formula, and then do:

$new\_weight = old\_weight-(gradient*learning\_rate)$

This gives us some weights that should decrease our cost function on the next iteration (at least they would for the sample we just ran), and so we repeat the cycle with them.

<img src="https://raw.githubusercontent.com/pekoto/fast.ai/master/images/gd2.jpg" width=600 height=450>

Once we have ran through all of our data, we have completed one epoch of training, and we can calculate the RMSE for our entire data set, and start another epoch.

<img src="https://raw.githubusercontent.com/pekoto/fast.ai/master/images/gd3.jpg" width=600 height=450>


### Gradient Descent as Circuit/Chain Rule

Ref: https://www.youtube.com/watch?v=i94OvYb6noo

It's been suggested that backpropagation is just "keep applying the chain rule". This video has a good explanation and examples.

During the forward pass, we also store the derivative of each of our function "gates", then, during the backprop, we multiply the previous gradient by the gradient for this gate, and so on. This lets us run backprop with around the same efficiency as the forward pass. It would be far too complex to try and compute a huge derivative for the entire function, so we do it bit by bit instead. In fact, if you look at PyTorch, a lot of it consists of these "gate" modules.

<img src="https://raw.githubusercontent.com/pekoto/fast.ai/master/images/gd04.jpg" width=650 height=500>

